In [69]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import re
from sklearn.model_selection import train_test_split
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from string import digits,punctuation
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
nltk.download()

In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
df=pd.read_csv('Chats.csv')

In [16]:
pd.set_option('Max_colwidth',-1)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,Unnamed: 0,Text,Clean
0,0,What advantage is there in booking directly with an airline rather than through an agent? I have almost always booked with the airline but now have an agent whose price is around $50 cheaper than airline and airline does not have price match.,NaN
1,1,"Hi, Could someone please confirm if CX 884 - HKG-LAX business class seats are truly lie flat seats? The aircraft code shows 773, travel agent says they are lie flat but Seatguru shows them as recliners. Help is appreciated. Cheers",NaN
2,2,I will be transiting Dubai soon en route to Oz. Never flown anywhere via Dubai before so am a bit wary about alcohol......not drinking it but having it in hand luggage. If I buy a bottle after security screening at LHR will it be a problem at any screening at Dubai before my connecting flight? It is a special gift requested by a friend in Oz.....but don't want to give myself any bother at airports...they are stressful enough places. Any advice?,NaN
3,3,"Does anyone know where I'd find estimated prices from Atlanta. My vacation destination is up in the air at this point and I'm flexible so I'd like to find a listing of places that are cheap to fly. Mexico, carribean, central american are all good choices. Thanks in advance.",NaN
4,4,It's from BA and finds the cheapest BA flight on a route britishairways.com/travel/…en_gb,NaN


In [20]:
df.Text.isnull().sum()

155

In [21]:
df.Text[df.Text.isnull()]

1947     NaN
1948     NaN
1949     NaN
1950     NaN
1951     NaN
        ... 
78483    NaN
78484    NaN
78485    NaN
78486    NaN
78487    NaN
Name: Text, Length: 155, dtype: object

In [ ]:
rows_to_remove=[row for row in range(len(df)) if df.iloc[row].isnull()['Text']==True]
len(rows_to_remove)

In [44]:
df.drop(rows_to_remove,axis=0,inplace=True)

In [46]:
X_train,X_test=train_test_split(df, test_size=0.4, random_state=0)

In [52]:
X_train.drop('Unnamed: 0',axis=1,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [57]:
X_train.set_index(pd.Index(range(X_train.shape[0])),inplace=True)


In [87]:
stemmer=PorterStemmer()
remove_digits = str.maketrans('', '', digits)

def tokenizer(text):
    text=text.translate(remove_digits)
    text=text.lower()
    text=re.sub(r"[^\w\s]",' ',text)
    token=nltk.word_tokenize(text)
    words=[stemmer.stem(word) for word in token if not word in stopwords.words('english') and not word in punctuation and not word in ['cname','name']]
    return (words)

In [88]:
vectorizer=CountVectorizer(tokenizer=tokenizer,max_features=10000)
vectors=vectorizer.fit_transform(X_train.Text)

In [89]:
vectors.A

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [90]:
lda=LatentDirichletAllocation(n_components=8,max_iter=3,learning_method='online',n_jobs=-1,random_state=0)

In [91]:
W1=lda.fit_transform(vectors)
H1=lda.components_

In [92]:
W1

array([[0.13312571, 0.1106428 , 0.00695191, ..., 0.00694514, 0.36135819,
        0.36707212],
       [0.00595426, 0.00595455, 0.00596421, ..., 0.00595434, 0.9583071 ,
        0.00595306],
       [0.78119121, 0.03125748, 0.03125345, ..., 0.03125   , 0.03127098,
        0.03127688],
       ...,
       [0.44293666, 0.01787636, 0.01786692, ..., 0.01786026, 0.01786375,
        0.01786775],
       [0.69580787, 0.01250155, 0.12110144, ..., 0.01252176, 0.01251989,
        0.12054355],
       [0.00961952, 0.22442158, 0.38498803, ..., 0.23739873, 0.00962778,
        0.0096242 ]])

In [93]:
H1

array([[1.25000003e-01, 1.25057516e-01, 1.25131784e-01, ...,
        1.25000017e-01, 1.25042474e-01, 1.25016772e-01],
       [1.25000004e-01, 1.25098518e-01, 1.25083241e-01, ...,
        1.25000021e-01, 1.41763739e+02, 1.25029732e-01],
       [1.25110398e-01, 4.38501282e+02, 1.25144237e-01, ...,
        1.25000009e-01, 1.27263025e-01, 1.25007784e-01],
       ...,
       [5.81940577e+00, 1.25191315e-01, 1.25039382e-01, ...,
        1.25000020e-01, 1.25026445e-01, 6.93235416e+00],
       [1.25000002e-01, 1.25218442e-01, 1.25007228e-01, ...,
        1.25000009e-01, 1.25136209e-01, 1.25442096e-01],
       [1.25000003e-01, 1.25062742e-01, 1.79376436e+02, ...,
        5.07027236e+00, 1.25024619e-01, 1.25000137e-01]])

In [ ]:
vectorizer.get_feature_names()

In [95]:
top=10
vocab=np.array(vectorizer.get_feature_names())
top_words=lambda t: [vocab[i] for i in np.argsort(t)[:-top-1:-1]]
topic_words=[top_words(t) for t in H1]
topic=[",".join(t) for t in topic_words]

In [96]:
topic

['train,need,ticket,station,trip,leav,mile,mileag,plan,award',
 'servic,would,bill,internet,tv,like,go,custom,cost,pay',
 'seat,flight,book,airlin,fli,travel,thank,go,would,one',
 'email,account,get,tri,purchas,number,ticket,sign,password,work',
 'travel,still,address,problem,year,buy,abl,passport,use,date',
 'see,passeng,ask,thing,time,left,com,trip,need,order',
 'flight,check,time,hour,fli,would,airport,thank,week,bag',
 'chang,reserv,ticket,pay,card,credit,flight,book,number,cancel']

In [97]:
colnames=["Topic "+str(i) for i in range(lda.n_components)]
docs=["Doc"+str(i) for i in range(len(X_train.Text))]

In [113]:
df_topic=pd.DataFrame(np.round(W1,2),columns=colnames,index=docs)
main_topic=np.argmax(df_topic.values,axis=1)
df_topic['Main Topic']=main_topic


In [110]:
df_topic

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Main Topic
Doc0,0.13,0.11,0.01,0.01,0.01,0.01,0.36,0.37,7
Doc1,0.01,0.01,0.01,0.01,0.01,0.01,0.96,0.01,6
Doc2,0.78,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0
Doc3,0.02,0.87,0.02,0.02,0.02,0.02,0.02,0.02,1
Doc4,0.01,0.01,0.94,0.01,0.01,0.01,0.01,0.01,2
...,...,...,...,...,...,...,...,...,...
Doc48659,0.00,0.27,0.00,0.00,0.00,0.28,0.06,0.37,7
Doc48660,0.02,0.02,0.02,0.89,0.02,0.02,0.02,0.02,3
Doc48661,0.44,0.02,0.02,0.02,0.45,0.02,0.02,0.02,4
Doc48662,0.70,0.01,0.12,0.01,0.01,0.01,0.01,0.12,0
